# RAG ਵਰਤ ਕੇ Azure AI Agent Service ਅਤੇ Semantic Kernel

ਇਹ ਕੋਡ ਟੁਕੜਾ ਦਿਖਾਉਂਦਾ ਹੈ ਕਿ ਕਿਵੇਂ `Azure AI Agent Service` ਅਤੇ `Semantic Kernel` ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਰਿਟਰੀਵਲ-ਅਗਮੈਂਟਡ ਜਨਰੇਸ਼ਨ (RAG) ਲਈ ਇੱਕ Azure AI ਏਜੰਟ ਬਣਾਇਆ ਅਤੇ ਪ੍ਰਬੰਧਿਤ ਕੀਤਾ ਜਾ ਸਕਦਾ ਹੈ। ਏਜੰਟ ਵਰਤੋਂਕਾਰ ਦੇ ਪ੍ਰਸ਼ਨਾਂ ਨੂੰ ਪ੍ਰਾਪਤ ਕੀਤੇ ਸੰਦਰਭ ਦੇ ਆਧਾਰ 'ਤੇ ਪ੍ਰਕਿਰਿਆ ਕਰਦਾ ਹੈ ਅਤੇ ਇਸ ਦੇ ਅਨੁਸਾਰ ਸਹੀ ਜਵਾਬ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ।


ਸਕਲਾਈਟ ਵਰਜਨ ਫਿਕਸ  
ਜੇਕਰ ਤੁਹਾਨੂੰ ਇਹ ਗਲਤੀ ਆਉਂਦੀ ਹੈ:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

ਆਪਣੇ ਨੋਟਬੁੱਕ ਦੇ ਸ਼ੁਰੂ ਵਿੱਚ ਇਸ ਕੋਡ ਬਲਾਕ ਨੂੰ ਅਨਕਮੈਂਟ ਕਰੋ:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### ਪੈਕੇਜਾਂ ਇੰਪੋਰਟ ਕਰਨਾ  
ਹੇਠਾਂ ਦਿੱਤਾ ਕੋਡ ਲੋੜੀਂਦੇ ਪੈਕੇਜਾਂ ਨੂੰ ਇੰਪੋਰਟ ਕਰਦਾ ਹੈ:  


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# ਸੇਮਾਂਟਿਕ ਕਰਨਲ ਅਤੇ ਐਜ਼ਰ ਏਆਈ ਏਜੰਟ ਸੇਵਾ ਨਾਲ ਰੀਟਰੀਵਲ-ਅਗਮੈਂਟੇਡ ਜਨਰੇਸ਼ਨ

ਇਹ ਨਮੂਨਾ ਦਿਖਾਉਂਦਾ ਹੈ ਕਿ **ਐਜ਼ਰ ਏਆਈ ਏਜੰਟ ਸੇਵਾ** ਦੀ ਵਰਤੋਂ ਕਰਕੇ **ਰੀਟਰੀਵਲ-ਅਗਮੈਂਟੇਡ ਜਨਰੇਸ਼ਨ (RAG)** ਕਿਵੇਂ ਕੀਤੀ ਜਾ ਸਕਦੀ ਹੈ, ਜਿਸ ਵਿੱਚ ਭਾਸ਼ਾ ਮਾਡਲ ਨੂੰ ਅਪਲੋਡ ਕੀਤੇ ਗਏ ਦਸਤਾਵੇਜ਼ ਤੋਂ ਡੋਮੇਨ-ਵਿਸ਼ੇਸ਼ ਸੰਦਰਭ ਨਾਲ ਜੋੜਿਆ ਜਾਂਦਾ ਹੈ।

### ਇਹ ਕਿਵੇਂ ਕੰਮ ਕਰਦਾ ਹੈ

1. **ਦਸਤਾਵੇਜ਼ ਅਪਲੋਡ**: ਇੱਕ ਮਾਰਕਡਾਊਨ ਫਾਈਲ (document.md) ਜਿਸ ਵਿੱਚ ਜਾਣਕਾਰੀ (ਕੋਂਟੋਸੋ ਦੀ ਯਾਤਰਾ ਬੀਮਾ ਨੀਤੀ) ਸ਼ਾਮਲ ਹੈ, ਏਜੰਟ ਸੇਵਾ ਵਿੱਚ ਅਪਲੋਡ ਕੀਤੀ ਜਾਂਦੀ ਹੈ।

2. **ਵੈਕਟਰ ਸਟੋਰ ਬਣਾਉਣਾ**: ਦਸਤਾਵੇਜ਼ ਨੂੰ ਵੈਕਟਰ ਸਟੋਰ ਵਿੱਚ ਇੰਡੈਕਸ ਕੀਤਾ ਜਾਂਦਾ ਹੈ ਤਾਂ ਜੋ ਇਸਦੇ ਸਮੱਗਰੀ 'ਤੇ ਸੇਮਾਂਟਿਕ ਖੋਜ ਕੀਤੀ ਜਾ ਸਕੇ।

3. **ਏਜੰਟ ਸੰਰਚਨਾ**: `gpt-4o` ਮਾਡਲ ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਇੱਕ ਏਜੰਟ ਬਣਾਇਆ ਜਾਂਦਾ ਹੈ ਜਿਸ ਵਿੱਚ ਹੇਠਾਂ ਦਿੱਤੀਆਂ ਸਖਤ ਹਦਾਇਤਾਂ ਸ਼ਾਮਲ ਹਨ:
   - ਸਿਰਫ਼ ਦਸਤਾਵੇਜ਼ ਤੋਂ ਪ੍ਰਾਪਤ ਕੀਤੀ ਸਮੱਗਰੀ ਦੇ ਆਧਾਰ 'ਤੇ ਸਵਾਲਾਂ ਦੇ ਜਵਾਬ ਦਿਓ।
   - ਜੇਕਰ ਸਵਾਲ ਵਿਸ਼ੇ ਤੋਂ ਬਾਹਰ ਹੋਵੇ ਤਾਂ ਜਵਾਬ ਦੇਣ ਤੋਂ ਇਨਕਾਰ ਕਰੋ।

4. **ਫਾਈਲ ਖੋਜ ਟੂਲ ਇੰਟੀਗ੍ਰੇਸ਼ਨ**: `FileSearchTool` ਨੂੰ ਏਜੰਟ ਨਾਲ ਰਜਿਸਟਰ ਕੀਤਾ ਜਾਂਦਾ ਹੈ, ਜਿਸ ਨਾਲ ਮਾਡਲ ਨੂੰ ਅਨੁਮਾਨ ਦੌਰਾਨ ਇੰਡੈਕਸ ਕੀਤੇ ਦਸਤਾਵੇਜ਼ ਤੋਂ ਸਬੰਧਤ ਟੁਕੜੇ ਖੋਜਣ ਅਤੇ ਪ੍ਰਾਪਤ ਕਰਨ ਦੀ ਸਹੂਲਤ ਮਿਲਦੀ ਹੈ।

5. **ਉਪਭੋਗਤਾ ਅੰਤਰਕਿਰਿਆ**: ਉਪਭੋਗਤਾ ਸਵਾਲ ਪੁੱਛ ਸਕਦੇ ਹਨ। ਜੇਕਰ ਦਸਤਾਵੇਜ਼ ਵਿੱਚ ਸਬੰਧਤ ਜਾਣਕਾਰੀ ਮਿਲਦੀ ਹੈ, ਤਾਂ ਏਜੰਟ ਇੱਕ ਮਜ਼ਬੂਤ ਜਵਾਬ ਤਿਆਰ ਕਰਦਾ ਹੈ।  
   ਜੇਕਰ ਨਹੀਂ, ਤਾਂ ਏਜੰਟ ਸਪਸ਼ਟ ਤੌਰ 'ਤੇ ਜਵਾਬ ਦਿੰਦਾ ਹੈ ਕਿ ਦਸਤਾਵੇਜ਼ ਵਿੱਚ ਕਾਫ਼ੀ ਜਾਣਕਾਰੀ ਨਹੀਂ ਹੈ।


### ਮੁੱਖ ਫੰਕਸ਼ਨ



ਪੱਕਾ ਕਰ ਲਵੋ ਕਿ `az login` ਨੂੰ Azure CLI ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਪਹਿਲਾਂ ਚਲਾਇਆ ਗਿਆ ਹੈ ਤਾਂ ਜੋ `DefaultAzureCredential` ਦੀ ਵਰਤੋਂ ਕਰਦੇ ਸਮੇਂ ਸਹੀ ਪ੍ਰਮਾਣਿਕਤਾ ਸੰਦਰਭ ਪ੍ਰਦਾਨ ਕੀਤਾ ਜਾ ਸਕੇ। Azure AI ਏਜੰਟ ਸੇਵਾ API ਕੁੰਜੀਆਂ ਦੀ ਵਰਤੋਂ ਨਹੀਂ ਕਰਦੀ।


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

ਸੋਧਣ ਲਈ ਕਿਰਪਾ ਕਰਕੇ ਮੈਨੂੰ ਉਹ ਮੂਲ ਟੈਕਸਟ ਦਿਓ ਜੋ ਤੁਸੀਂ ਪੰਜਾਬੀ ਵਿੱਚ ਅਨੁਵਾਦ ਕਰਵਾਉਣਾ ਚਾਹੁੰਦੇ ਹੋ।



---

**ਅਸਵੀਕਰਤੀ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦ ਕੀਤਾ ਗਿਆ ਹੈ। ਜਦੋਂ ਕਿ ਅਸੀਂ ਸਹੀ ਹੋਣ ਦਾ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਦਿਓ ਕਿ ਸਵੈਚਾਲਿਤ ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਸੁੱਤੀਆਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਇਸ ਦੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਮੌਜੂਦ ਮੂਲ ਦਸਤਾਵੇਜ਼ ਨੂੰ ਪ੍ਰਮਾਣਿਕ ਸਰੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਤੋਂ ਪੈਦਾ ਹੋਣ ਵਾਲੇ ਕਿਸੇ ਵੀ ਗਲਤਫਹਿਮੀ ਜਾਂ ਗਲਤ ਵਿਆਖਿਆ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।  
